# Project Stage I - Task 3 Part C
- Merge Enrichment Data (Employment Dataset) With Team Super File
### Francis Perez - Team 2

## Load Needed Imports

In [1]:
import pandas as pd
import numpy as np

## Load Data

In [2]:
#Load Team Super File From data folder
team_super_file = pd.read_csv("..\..\..\data\superCOVID-19datafame.csv")

#Load Employment Dataset
employment_dataset = pd.read_csv("..\..\..\data\EmploymentData_allhlcn201.csv")

### Clean Super File

In [3]:
#remove county fips with zeros
team_super_file = team_super_file[team_super_file.apply(lambda x: x["countyFIPS"] != 0, axis=1)]

#Make countyFIPS column a number
team_super_file["countyFIPS"] = team_super_file["countyFIPS"].apply(pd.to_numeric)

#Reindex super file now that we have removed the unneed counties
team_super_file.set_index("countyFIPS", inplace=True)

### Filter & Shape Employment data

In [4]:
#Only keep County Data
#Remove MSA, State, and Nation Rows
employment_dataset = employment_dataset[employment_dataset.apply(lambda x: x["Area Type"] == 'County', axis=1)]

#Combine Ownership & Industry column
employment_dataset["Employment Number Type"] = employment_dataset['Ownership'] + " " + employment_dataset['Industry']

#Filter employment number totals and keep the sub categories
employment_dataset = employment_dataset[employment_dataset.apply(lambda x:
    (x["Employment Number Type"] == "Federal Government 10 Total, all industries") or
    (x["Employment Number Type"] == "State Government 10 Total, all industries") or
    (x["Employment Number Type"] == "Local Government 10 Total, all industries") or
    (x["Employment Number Type"] == "Private 101 Goods-producing") or
    (x["Employment Number Type"] == "Private 102 Service-providing"), axis=1)]

#Revalue the "Employment Number Type" Column's values to give better label
employment_dataset.loc[(employment_dataset["Employment Number Type"] == "Federal Government 10 Total, all industries"), "Employment Number Type"] = "gov_fed_subtotal"
employment_dataset.loc[(employment_dataset["Employment Number Type"] == "State Government 10 Total, all industries"), "Employment Number Type"] = "gov_state_subtotal"
employment_dataset.loc[(employment_dataset["Employment Number Type"] == "Local Government 10 Total, all industries"), "Employment Number Type"] = "gov_local_subtotal"
employment_dataset.loc[(employment_dataset["Employment Number Type"] == "Private 101 Goods-producing"), "Employment Number Type"] = "priv_goods_subtotal"
employment_dataset.loc[(employment_dataset["Employment Number Type"] == "Private 102 Service-providing"), "Employment Number Type"] = "priv_services_subtotal"

employment_dataset.rename(columns={"Area\nCode":"countyFIPS"}, inplace=True)
employment_dataset["countyFIPS"] = employment_dataset["countyFIPS"].apply(pd.to_numeric)

#Drop non-needed Columns
employment_dataset.drop(columns=["St", "Cnty", "Own", "NAICS", 
    "St Name", "Total Wage Location Quotient Relative to U.S.", 
    "Employment Location Quotient Relative to U.S.", "Average Weekly Wage",
    "Establishment Count", "Status Code", "Total Quarterly Wages", "Area Type", 
    "Year", "Qtr", "Area", "Ownership", "Industry"], inplace=True)

In [5]:
#Show the employment dataset as is
employment_dataset.head()

,countyFIPS,January Employment,February Employment,March Employment,Employment Number Type
36,1001,81,84,85,gov_fed_subtotal
37,1001,579,586,573,gov_state_subtotal
38,1001,"1,903","1,925","1,940",gov_local_subtotal
40,1001,"2,320","2,241","2,224",priv_goods_subtotal
44,1001,"6,202","6,211","6,248",priv_services_subtotal


### Create January Numbers

In [6]:
#remove non-january months and place into new dataframe
employment_dataset_jan = employment_dataset.drop(columns=["February Employment", "March Employment"])

#clean strings remove commas
employment_dataset_jan["January Employment"] = employment_dataset_jan["January Employment"].str.replace(",","")

#convert col of strings to number
employment_dataset_jan["January Employment"] = pd.to_numeric(employment_dataset_jan["January Employment"])

#Create pivot on January's employment values
employment_dataset_jan = pd.DataFrame(employment_dataset_jan.pivot_table(index="countyFIPS", columns="Employment Number Type", values="January Employment",aggfunc=np.sum).reset_index())

#reindex data
employment_dataset_jan.reset_index(drop=True, inplace=True)
employment_dataset_jan.set_index("countyFIPS", inplace=True)

#rename January's columns
employment_dataset_jan.rename(columns={"gov_fed_subtotal":"employment_jan_gov_fed_subtotal",
                                        "gov_local_subtotal":"employment_jan_gov_local_subtotal",
                                        "gov_state_subtotal":"employment_jan_gov_state_subtotal",
                                        "priv_goods_subtotal":"employment_jan_priv_goods_subtotal",
                                        "priv_services_subtotal":"employment_jan_priv_services_subtotal"}, inplace=True)


#Create new column with total for the month
employment_dataset_jan["employment_jan_total"] = employment_dataset_jan["employment_jan_gov_fed_subtotal"] + \
                                                employment_dataset_jan["employment_jan_gov_local_subtotal"] + \
                                                employment_dataset_jan["employment_jan_gov_state_subtotal"] + \
                                                employment_dataset_jan["employment_jan_priv_goods_subtotal"] + \
                                                employment_dataset_jan["employment_jan_priv_services_subtotal"]

employment_dataset_jan.head(100)


Employment Number Type,employment_jan_gov_fed_subtotal,employment_jan_gov_local_subtotal,employment_jan_gov_state_subtotal,employment_jan_priv_goods_subtotal,employment_jan_priv_services_subtotal,employment_jan_total
countyFIPS,,,,,,
1001,81.0,1903.0,579.0,2320.0,6202.0,11085.0
1003,351.0,8556.0,1070.0,9201.0,54837.0,74015.0
1005,55.0,969.0,621.0,2954.0,3760.0,8359.0
1007,66.0,972.0,248.0,1551.0,1965.0,4802.0
1009,87.0,1861.0,162.0,1853.0,4497.0,8460.0
...,...,...,...,...,...,...
2275,37.0,166.0,15.0,38.0,491.0,747.0
2282,23.0,0.0,0.0,0.0,0.0,23.0
2290,56.0,1290.0,74.0,46.0,573.0,2039.0


### Create February Numbers

In [7]:
#remove non-february months and place into new dataframe
employment_dataset_feb = employment_dataset.drop(columns=["January Employment", "March Employment"])

#clean strings remove commas
employment_dataset_feb["February Employment"] = employment_dataset_feb["February Employment"].str.replace(",","")

#convert col of strings to number
employment_dataset_feb["February Employment"] = pd.to_numeric(employment_dataset_feb["February Employment"])

#Create pivot on February's employment values
employment_dataset_feb = pd.DataFrame(employment_dataset_feb.pivot_table(index="countyFIPS", columns="Employment Number Type", values="February Employment",aggfunc=np.sum).reset_index())

#reindex data
employment_dataset_feb.reset_index(drop=True, inplace=True)
employment_dataset_feb.set_index("countyFIPS", inplace=True)

#rename February's columns
employment_dataset_feb.rename(columns={"gov_fed_subtotal":"employment_feb_gov_fed_subtotal",
                                        "gov_local_subtotal":"employment_feb_gov_local_subtotal",
                                        "gov_state_subtotal":"employment_feb_gov_state_subtotal",
                                        "priv_goods_subtotal":"employment_feb_priv_goods_subtotal",
                                        "priv_services_subtotal":"employment_feb_priv_services_subtotal"}, inplace=True)


#Create new column with total for the month
employment_dataset_feb["employment_feb_total"] = employment_dataset_feb["employment_feb_gov_fed_subtotal"] + \
                                                employment_dataset_feb["employment_feb_gov_local_subtotal"] + \
                                                employment_dataset_feb["employment_feb_gov_state_subtotal"] + \
                                                employment_dataset_feb["employment_feb_priv_goods_subtotal"] + \
                                                employment_dataset_feb["employment_feb_priv_services_subtotal"]

employment_dataset_feb.head(100)

Employment Number Type,employment_feb_gov_fed_subtotal,employment_feb_gov_local_subtotal,employment_feb_gov_state_subtotal,employment_feb_priv_goods_subtotal,employment_feb_priv_services_subtotal,employment_feb_total
countyFIPS,,,,,,
1001,84.0,1925.0,586.0,2241.0,6211.0,11047.0
1003,349.0,8581.0,1219.0,9252.0,55942.0,75343.0
1005,53.0,987.0,616.0,2961.0,3779.0,8396.0
1007,63.0,990.0,242.0,1523.0,2001.0,4819.0
1009,85.0,1876.0,163.0,1905.0,4488.0,8517.0
...,...,...,...,...,...,...
2275,37.0,170.0,16.0,51.0,490.0,764.0
2282,23.0,0.0,0.0,0.0,0.0,23.0
2290,53.0,1338.0,73.0,52.0,584.0,2100.0


### Create March Numbers

In [8]:
#remove non-march months and place into new dataframe
employment_dataset_mar = employment_dataset.drop(columns=["January Employment", "February Employment"])

#clean strings remove commas
employment_dataset_mar["March Employment"] = employment_dataset_mar["March Employment"].str.replace(",","")

#convert col of strings to number
employment_dataset_mar["March Employment"] = pd.to_numeric(employment_dataset_mar["March Employment"])

#Create pivot on March's employment values
employment_dataset_mar = pd.DataFrame(employment_dataset_mar.pivot_table(index="countyFIPS", columns="Employment Number Type", values="March Employment",aggfunc=np.sum).reset_index())

#reindex data
employment_dataset_mar.reset_index(drop=True, inplace=True)
employment_dataset_mar.set_index("countyFIPS", inplace=True)

#rename March's columns
employment_dataset_mar.rename(columns={"gov_fed_subtotal":"employment_mar_gov_fed_subtotal",
                                        "gov_local_subtotal":"employment_mar_gov_local_subtotal",
                                        "gov_state_subtotal":"employment_mar_gov_state_subtotal",
                                        "priv_goods_subtotal":"employment_mar_priv_goods_subtotal",
                                        "priv_services_subtotal":"employment_mar_priv_services_subtotal"}, inplace=True)


#Create new column with total for the month
employment_dataset_mar["employment_mar_total"] = employment_dataset_mar["employment_mar_gov_fed_subtotal"] + \
                                                employment_dataset_mar["employment_mar_gov_local_subtotal"] + \
                                                employment_dataset_mar["employment_mar_gov_state_subtotal"] + \
                                                employment_dataset_mar["employment_mar_priv_goods_subtotal"] + \
                                                employment_dataset_mar["employment_mar_priv_services_subtotal"]

employment_dataset_mar.head(100)

Employment Number Type,employment_mar_gov_fed_subtotal,employment_mar_gov_local_subtotal,employment_mar_gov_state_subtotal,employment_mar_priv_goods_subtotal,employment_mar_priv_services_subtotal,employment_mar_total
countyFIPS,,,,,,
1001,85.0,1940.0,573.0,2224.0,6248.0,11070.0
1003,364.0,8690.0,1211.0,9262.0,56534.0,76061.0
1005,55.0,985.0,623.0,2949.0,3798.0,8410.0
1007,66.0,999.0,245.0,1553.0,2005.0,4868.0
1009,91.0,1880.0,164.0,1935.0,4490.0,8560.0
...,...,...,...,...,...,...
2275,38.0,166.0,16.0,58.0,486.0,764.0
2282,23.0,0.0,0.0,0.0,0.0,23.0
2290,63.0,1333.0,78.0,49.0,571.0,2094.0


### Merge January Dataframes To Team Super File

In [9]:
#merge super file and January, with January on the right ending of the columms
merged = pd.merge(left=team_super_file, right=employment_dataset_jan, left_index=True, right_index=True, left_on="countyFIPS", right_on="countyFIPS" , how="inner")
merged.head()

,County Name,State,population,stateFIPS,confirmed_1/22/20,confirmed_1/23/20,confirmed_1/24/20,confirmed_1/25/20,confirmed_1/26/20,confirmed_1/27/20,...,deaths_10/15/20,deaths_10/16/20,deaths_10/17/20,deaths_10/18/20,employment_jan_gov_fed_subtotal,employment_jan_gov_local_subtotal,employment_jan_gov_state_subtotal,employment_jan_priv_goods_subtotal,employment_jan_priv_services_subtotal,employment_jan_total
countyFIPS,,,,,,,,,,,,,,,,,,,,,
1001,Autauga County,AL,55869,1,0,0,0,0,0,0,...,28,28,28,28,81.0,1903.0,579.0,2320.0,6202.0,11085.0
1003,Baldwin County,AL,223234,1,0,0,0,0,0,0,...,66,67,67,67,351.0,8556.0,1070.0,9201.0,54837.0,74015.0
1005,Barbour County,AL,24686,1,0,0,0,0,0,0,...,9,9,9,9,55.0,969.0,621.0,2954.0,3760.0,8359.0
1007,Bibb County,AL,22394,1,0,0,0,0,0,0,...,13,13,13,13,66.0,972.0,248.0,1551.0,1965.0,4802.0
1009,Blount County,AL,57826,1,0,0,0,0,0,0,...,23,23,23,23,87.0,1861.0,162.0,1853.0,4497.0,8460.0


### Merge February Dataframe

In [10]:
#merge February data into the new merged File.
merged = pd.merge(left=merged, right=employment_dataset_feb, left_index=True, right_index=True, left_on="countyFIPS", right_on="countyFIPS" , how="inner")
merged.head()

,County Name,State,population,stateFIPS,confirmed_1/22/20,confirmed_1/23/20,confirmed_1/24/20,confirmed_1/25/20,confirmed_1/26/20,confirmed_1/27/20,...,employment_jan_gov_state_subtotal,employment_jan_priv_goods_subtotal,employment_jan_priv_services_subtotal,employment_jan_total,employment_feb_gov_fed_subtotal,employment_feb_gov_local_subtotal,employment_feb_gov_state_subtotal,employment_feb_priv_goods_subtotal,employment_feb_priv_services_subtotal,employment_feb_total
countyFIPS,,,,,,,,,,,,,,,,,,,,,
1001,Autauga County,AL,55869,1,0,0,0,0,0,0,...,579.0,2320.0,6202.0,11085.0,84.0,1925.0,586.0,2241.0,6211.0,11047.0
1003,Baldwin County,AL,223234,1,0,0,0,0,0,0,...,1070.0,9201.0,54837.0,74015.0,349.0,8581.0,1219.0,9252.0,55942.0,75343.0
1005,Barbour County,AL,24686,1,0,0,0,0,0,0,...,621.0,2954.0,3760.0,8359.0,53.0,987.0,616.0,2961.0,3779.0,8396.0
1007,Bibb County,AL,22394,1,0,0,0,0,0,0,...,248.0,1551.0,1965.0,4802.0,63.0,990.0,242.0,1523.0,2001.0,4819.0
1009,Blount County,AL,57826,1,0,0,0,0,0,0,...,162.0,1853.0,4497.0,8460.0,85.0,1876.0,163.0,1905.0,4488.0,8517.0


### Merge March Dataframe

In [11]:
#merge March data into the new merged File.
merged = pd.merge(left=merged, right=employment_dataset_mar, left_index=True, right_index=True, left_on="countyFIPS", right_on="countyFIPS" , how="inner")
merged.head()

,County Name,State,population,stateFIPS,confirmed_1/22/20,confirmed_1/23/20,confirmed_1/24/20,confirmed_1/25/20,confirmed_1/26/20,confirmed_1/27/20,...,employment_feb_gov_state_subtotal,employment_feb_priv_goods_subtotal,employment_feb_priv_services_subtotal,employment_feb_total,employment_mar_gov_fed_subtotal,employment_mar_gov_local_subtotal,employment_mar_gov_state_subtotal,employment_mar_priv_goods_subtotal,employment_mar_priv_services_subtotal,employment_mar_total
countyFIPS,,,,,,,,,,,,,,,,,,,,,
1001,Autauga County,AL,55869,1,0,0,0,0,0,0,...,586.0,2241.0,6211.0,11047.0,85.0,1940.0,573.0,2224.0,6248.0,11070.0
1003,Baldwin County,AL,223234,1,0,0,0,0,0,0,...,1219.0,9252.0,55942.0,75343.0,364.0,8690.0,1211.0,9262.0,56534.0,76061.0
1005,Barbour County,AL,24686,1,0,0,0,0,0,0,...,616.0,2961.0,3779.0,8396.0,55.0,985.0,623.0,2949.0,3798.0,8410.0
1007,Bibb County,AL,22394,1,0,0,0,0,0,0,...,242.0,1523.0,2001.0,4819.0,66.0,999.0,245.0,1553.0,2005.0,4868.0
1009,Blount County,AL,57826,1,0,0,0,0,0,0,...,163.0,1905.0,4488.0,8517.0,91.0,1880.0,164.0,1935.0,4490.0,8560.0


### Export file to csv with file named Super_Employment_Merged_Task3.cvs

In [12]:
#Export file to this folder
merged.to_csv("..\..\..\data\Francis\Super_Employment_Merged_Task3.csv")